In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
raw_data = pd.read_csv("D:\Coding\Machine Learning\Bioinformatics\QSAR study on falcipain inhibitor\Dataset Falciapin\Data_Sheet_1_In_silico_Guided_Drug_Repurposing_Discovery_of_New.CSV", sep=";")
raw_data

,Compound,SMILE,FP-2 IC50 (uM),FP-2 inhibition rate at 10 uM (%),FP-2 inhibition rate at 20 uM (%)
0,2,OCCCc1sc(\C=C\C(=O)CSCCCc2ccccc2)nc1CCc1ccccc1,6.6,NaN,NaN
1,3,OCCCc1sc(CCC(=O)CSCCCc2ccccc2)nc1CCc1ccccc1,>12.5,NaN,NaN
2,4,O=CCCc1sc(CCC(=O)CSCCCc2ccccc2)nc1CCc1ccccc1,39.1,NaN,NaN
3,5,CC(=O)OCc1nc(CCC(=O)CSCCCc2ccccc2)sc1CCc1ccccc1,>33,NaN,NaN
4,6,NC(=O)CCc1sc(CCC(=O)CSCCCc2ccccc2)nc1CCc1ccccc1,>33,NaN,NaN
...,...,...,...,...,...
510,575,Oc1cc(O)c2c(c1)oc(-c1cc(O)c(O)c(O)c1)c(O)c2=O,0.1,76.4,NaN
511,576,Oc1cc(O)c2c(c1)oc(-c1ccc(O)c(O)c1)c(O)c2=O,0.3,75.8,NaN
512,577,Oc1ccc(c(O)c1)-c1oc2cc(O)cc(O)c2c(=O)c1O,0.2,79.9,NaN
513,578,O[C@@H]1[C@H](Oc2cc(O)cc(O)c2C1=O)c1ccc(O)c(O)c1,NaN,16.9,NaN


In [83]:
raw_data.isna().sum()

Compound                               0
SMILE                                  0
FP-2 IC50 (uM)                        66
FP-2 inhibition rate at 10 uM (%)    422
FP-2 inhibition rate at 20 uM (%)    502
dtype: int64

In [84]:
raw_data.dtypes

Compound                               int64
SMILE                                 object
FP-2 IC50 (uM)                        object
FP-2 inhibition rate at 10 uM (%)     object
FP-2 inhibition rate at 20 uM (%)    float64
dtype: object

### Cleaning Raw Data

In [3]:
smiles_ic50 = raw_data.drop(["FP-2 inhibition rate at 10 uM (%)", "FP-2 inhibition rate at 20 uM (%)"], axis=1)
smiles_ic50

,Compound,SMILE,FP-2 IC50 (uM)
0,2,OCCCc1sc(\C=C\C(=O)CSCCCc2ccccc2)nc1CCc1ccccc1,6.6
1,3,OCCCc1sc(CCC(=O)CSCCCc2ccccc2)nc1CCc1ccccc1,>12.5
2,4,O=CCCc1sc(CCC(=O)CSCCCc2ccccc2)nc1CCc1ccccc1,39.1
3,5,CC(=O)OCc1nc(CCC(=O)CSCCCc2ccccc2)sc1CCc1ccccc1,>33
4,6,NC(=O)CCc1sc(CCC(=O)CSCCCc2ccccc2)nc1CCc1ccccc1,>33
...,...,...,...
510,575,Oc1cc(O)c2c(c1)oc(-c1cc(O)c(O)c(O)c1)c(O)c2=O,0.1
511,576,Oc1cc(O)c2c(c1)oc(-c1ccc(O)c(O)c1)c(O)c2=O,0.3
512,577,Oc1ccc(c(O)c1)-c1oc2cc(O)cc(O)c2c(=O)c1O,0.2
513,578,O[C@@H]1[C@H](Oc2cc(O)cc(O)c2C1=O)c1ccc(O)c(O)c1,NaN


In [4]:
smiles_ic50.isna().sum()

Compound           0
SMILE              0
FP-2 IC50 (uM)    66
dtype: int64

In [5]:
smiles_ic50_no_nan = smiles_ic50.dropna(axis=0, how='any')
smiles_ic50_no_nan

,Compound,SMILE,FP-2 IC50 (uM)
0,2,OCCCc1sc(\C=C\C(=O)CSCCCc2ccccc2)nc1CCc1ccccc1,6.6
1,3,OCCCc1sc(CCC(=O)CSCCCc2ccccc2)nc1CCc1ccccc1,>12.5
2,4,O=CCCc1sc(CCC(=O)CSCCCc2ccccc2)nc1CCc1ccccc1,39.1
3,5,CC(=O)OCc1nc(CCC(=O)CSCCCc2ccccc2)sc1CCc1ccccc1,>33
4,6,NC(=O)CCc1sc(CCC(=O)CSCCCc2ccccc2)nc1CCc1ccccc1,>33
...,...,...,...
509,574,Oc1ccc(cc1)-c1oc2cc(O)cc(O)c2c(=O)c1O,0.2
510,575,Oc1cc(O)c2c(c1)oc(-c1cc(O)c(O)c(O)c1)c(O)c2=O,0.1
511,576,Oc1cc(O)c2c(c1)oc(-c1ccc(O)c(O)c1)c(O)c2=O,0.3
512,577,Oc1ccc(c(O)c1)-c1oc2cc(O)cc(O)c2c(=O)c1O,0.2


In [12]:
smiles_ic50_no_nan.loc[smiles_ic50_no_nan["FP-2 IC50 (uM)"].str.contains('>|-', regex=True)]

,Compound,SMILE,FP-2 IC50 (uM)
1,3,OCCCc1sc(CCC(=O)CSCCCc2ccccc2)nc1CCc1ccccc1,>12.5
3,5,CC(=O)OCc1nc(CCC(=O)CSCCCc2ccccc2)sc1CCc1ccccc1,>33
4,6,NC(=O)CCc1sc(CCC(=O)CSCCCc2ccccc2)nc1CCc1ccccc1,>33
6,9,OCc1nc(CCC(=O)CSCCCc2ccccc2)sc1CCc1ccccc1,>12.56
7,10,CC(=O)NCCCc1sc(CCC(=O)CSCCCc2ccccc2)nc1CCc1ccccc1,>33
...,...,...,...
497,561,CC(=O)c1ccc(NC(=O)CSc2nnc(COc3ccc4cccnc4c3)n2-...,>50
498,562,COc1ccc(cc1)C(=O)NNC(=O)CSc1nnc(COc2ccc3cccnc3...,>50
499,563,COc1cccc(\C=N\NC(=O)C2=NN(Cc3ccc(Cl)cc3Cl)CC2)c1O,>50
500,564,CC(NC(=O)COc1ccc2ccccc2c1)C(=O)N\N=C\c1cccc(Cl...,>50


In [17]:
print('Total rows without relative values:', 449 - 135)

Total rows without relative values: 314


### Final SMILES dan IC50 data

In [35]:
smiles_ic50_cleaned = smiles_ic50_no_nan.drop(smiles_ic50_no_nan[smiles_ic50_no_nan["FP-2 IC50 (uM)"].str.contains('>|-', regex=True)].index, axis=0, inplace=False)
smiles_ic50_cleaned = smiles_ic50_cleaned.reset_index(drop=True)
smiles_ic50_cleaned['FP-2 IC50 (uM)'] = smiles_ic50_cleaned['FP-2 IC50 (uM)'].astype('float')
smiles_ic50_cleaned

,Compound,SMILE,FP-2 IC50 (uM)
0,2,OCCCc1sc(\C=C\C(=O)CSCCCc2ccccc2)nc1CCc1ccccc1,6.60
1,4,O=CCCc1sc(CCC(=O)CSCCCc2ccccc2)nc1CCc1ccccc1,39.10
2,8,NC(=O)CCc1sc(NC(=O)CSCCCc2ccccc2)nc1\C=C\c1ccccc1,15.30
3,14,CCCC\C(=N/NC(N)=S)c1ccc(Cl)c(Cl)c1,0.62
4,15,CC(C)(C)C(=N/NC(N)=S)\c1ccc(Cl)c(Cl)c1,0.50
...,...,...,...
309,574,Oc1ccc(cc1)-c1oc2cc(O)cc(O)c2c(=O)c1O,0.20
310,575,Oc1cc(O)c2c(c1)oc(-c1cc(O)c(O)c(O)c1)c(O)c2=O,0.10
311,576,Oc1cc(O)c2c(c1)oc(-c1ccc(O)c(O)c1)c(O)c2=O,0.30
312,577,Oc1ccc(c(O)c1)-c1oc2cc(O)cc(O)c2c(=O)c1O,0.20


In [36]:
smiles_ic50_cleaned.dtypes

Compound            int64
SMILE              object
FP-2 IC50 (uM)    float64
dtype: object

In [37]:
smiles_ic50_cleaned['FP-2 pIC50 (uM)'] = smiles_ic50_cleaned['FP-2 IC50 (uM)']*100

In [38]:
x

0       660.0
1      3910.0
2      1530.0
3        62.0
4        50.0
        ...  
309      20.0
310      10.0
311      30.0
312      20.0
313      30.0
Name: FP-2 IC50 (uM), Length: 314, dtype: float64